In [ ]:
!pip install yfinance

In [ ]:
!pip install talib-binary

In [ ]:
#取得歷史報價並畫圖
def get_data(stock,start,end):
    #加入取得報價的套件
    import yfinance as yf
    
    #歷史報價存入df(查詢標的,開始時間,結束時間)
    df = yf.download(stock,start,end,interval = "1d").dropna()#download data from yahoo
    #interval=1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    #period=1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    
    #畫圖
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.figure(figsize=(20,5))
    plt.title(stock+' Adj Close')
    plt.plot(df['Adj Close'])
    plt.show()
    
    return df

#整理資料與增加複雜度
def sort_data(df):
    df['Open'] = df.Open
    df['High'] = df.High
    df['percent_change'] = df['Close'].pct_change()
    #df['std_5'] = df['percent_change'].rolling(5).std()
    #df['ret_5'] = df['percent_change'].rolling(5).mean()
    #df.dropna(inplace=True)
    return df

def split_data(df):
    #label
    # X is the input variable
    X = df[['Open', 'High', 'Adj Close', 'Volume']]

    # Y is the target or output variable
    y = np.where(df['percent_change'].shift(-1) > 0, 1, 0)
    #y = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

    #分割訓練組跟測試組
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    print(X_train.shape, X_test.shape)
    print(y_train.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

def DecisionTreeClassifier(X_train, X_test, y_train, y_test):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    #clf = RandomForestClassifier(n_estimators=1000, random_state=10, n_jobs=-1, min_samples_leaf = 5)
    # Create the model on train dataset

    # Decision Tree
    clf = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=10)

    model = clf.fit(X_train, y_train)
    y_pre = clf.predict(X_test)
    
    from sklearn.metrics import classification_report
    report = classification_report(y_test, y_pre)
    print(report)
    print('DecisionTreeClassifier')
    return y_test, y_pre

def auc_roc(y_test, y_pre):  
    from sklearn import metrics
    # 績效
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pre)
    auc = metrics.auc(fpr, tpr)
    print(auc)
    import matplotlib.pyplot as plt
    
    plt.plot(fpr,tpr,label="decision tree, auc="+str(auc),color='blue')
    plt.title("ROC")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc=4)
    plt.show()
    
import numpy as np
import pandas as pd
stock='^TWII'#股票種類
df=get_data(stock,'2010-04-27','2021-04-27')#取得歷史報價並畫圖
df=sort_data(df)
df.head(10)
X_train, X_test, y_train, y_test=split_data(df)
y_test_RF, y_pre_RF=DecisionTreeClassifier(X_train, X_test, y_train, y_test)

auc_roc(y_test, y_pre_RF)